In [13]:
import numpy as np
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import SimpleRNN,LSTM, Dense, Activation
from keras.utils import to_categorical
#from sklearn.preprocessing import OneHotEncoder

In [14]:
A01T = h5py.File('datasets/A01T_slice.mat','r')
data = np.copy(A01T['image'])
data = np.transpose(data,(0,2,1))
data = data[:,:,:22]
labels = np.copy(A01T['type'])
labels = labels[0,0:data.shape[0]:1]
labels = np.asarray(labels, dtype=np.int32)

#enc = OneHotEncoder()
#enc_labels = enc.fit_transform(labels.reshape(-1,1)).toarray()
enc_labels = to_categorical(labels-769, num_classes=4)
print(enc_labels)

[[ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 ..., 
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]


In [4]:
model = Sequential([
    LSTM(100, input_shape=(1000, 22)),
    Dense(32),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(4),
    Activation('softmax'),
])

bs, t, f = data.shape

In [5]:
model.layers

In [7]:
shuffle = np.random.choice(bs,bs,replace=False)

train_samples = 200
train_data = data[:train_samples,:,:]
train_labels = enc_labels[:train_samples]
test_data = data[train_samples:,:,:]
test_labels =enc_labels[train_samples:]

model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])

hist = model.fit(train_data,train_labels,epochs=5,validation_split=0.25,batch_size=32)
test_score = model.evaluate(test_data, test_labels, batch_size=32)


Train on 150 samples, validate on 50 samples
Epoch 1/5
150/150 [==============================] - 25s 164ms/step - loss: nan - acc: 0.2333 - val_loss: nan - val_acc: 0.2600
Epoch 2/5
150/150 [==============================] - 37s 246ms/step - loss: nan - acc: 0.2533 - val_loss: nan - val_acc: 0.2600
Epoch 3/5
150/150 [==============================] - 39s 259ms/step - loss: nan - acc: 0.2533 - val_loss: nan - val_acc: 0.2600
Epoch 4/5
150/150 [==============================] - 27s 181ms/step - loss: nan - acc: 0.2533 - val_loss: nan - val_acc: 0.2600
Epoch 5/5
88/88 [==============================] - 8s 92ms/step


In [8]:
test_score

[nan, 0.23863636363636365]

Modified VGG net to handle our input (need to check dimensions and might need to transpose input to original shape)

In [9]:
# VGGnet
# https://keras.io/getting-started/sequential-model-guide/#examples

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.optimizers import SGD

del model
model = Sequential()
#model.add(LSTM(100, input_shape=(1000, 22)))
model.add(Conv1D(512, 3, activation='relu',input_shape=(1000, 22)))             #Originally 32 each
model.add(Conv1D(512, 3, activation='relu'))
model.add(MaxPooling1D())
model.add(Dropout(0.25))

model.add(Conv1D(64, 3, activation='relu'))                                    #Originally 64 each
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])

hist = model.fit(train_data,train_labels,epochs=5,validation_split=0.25,batch_size=32)
test_score = model.evaluate(test_data, test_labels, batch_size=32)

Train on 150 samples, validate on 50 samples
Epoch 1/5
150/150 [==============================] - 30s 199ms/step - loss: nan - acc: 0.2733 - val_loss: nan - val_acc: 0.2600
Epoch 2/5
150/150 [==============================] - 24s 161ms/step - loss: nan - acc: 0.2533 - val_loss: nan - val_acc: 0.2600
Epoch 3/5
150/150 [==============================] - 33s 219ms/step - loss: nan - acc: 0.2533 - val_loss: nan - val_acc: 0.2600
Epoch 4/5
150/150 [==============================] - 35s 235ms/step - loss: nan - acc: 0.2533 - val_loss: nan - val_acc: 0.2600
Epoch 5/5
88/88 [==============================] - 4s 46ms/step


In [10]:
test_score

[nan, 0.23863636363636365]

Simple RNN model

In [11]:

model = Sequential([
    SimpleRNN(64, input_shape=(t,f)),
    Dense(32),
    Activation('relu'),
    Dense(4),
    Activation('softmax'),
])

model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])

hist = model.fit(train_data,train_labels,epochs=5,validation_split=0.25,batch_size=32)
test_score = model.evaluate(test_data, test_labels, batch_size=32)

Train on 150 samples, validate on 50 samples
Epoch 1/5
150/150 [==============================] - 2s 13ms/step - loss: 1.6417 - acc: 0.2533 - val_loss: 1.6417 - val_acc: 0.2800
Epoch 2/5
150/150 [==============================] - 1s 8ms/step - loss: 1.5290 - acc: 0.2333 - val_loss: 1.4297 - val_acc: 0.2200
Epoch 3/5
150/150 [==============================] - 1s 8ms/step - loss: 1.4027 - acc: 0.2533 - val_loss: 1.4433 - val_acc: 0.2600
Epoch 4/5
150/150 [==============================] - 1s 9ms/step - loss: 1.4271 - acc: 0.2533 - val_loss: 1.4319 - val_acc: 0.2600
Epoch 5/5
88/88 [==============================] - 0s 3ms/step


In [15]:
test_score

[1.3988697637211194, 0.23863636363636365]